In [31]:
## Install libraries as needed
# There are many libraries for similarity (Levenstein) like editdistance and enchant
## For editdistance, 
#!pip install editdistance
## For enchant, 
#!pip install pyenchant

In [1]:
#importing necessary libs
import json
import editdistance #lib for computing levenshtein distance
import ast
from operator import itemgetter

In [2]:
#loading the cases.json file
with open("../../cases/cases.json") as f:
    file = json.load(f)

In [3]:
#function for taking in user input and creating the representation for distance calculation methods
def caseRepnGen(inp_dict):
    res = ast.literal_eval(inp_dict)
    domain = res["domain"]
    init = res["init"]
    goal = res["goal"]
    return domain, "|".join(init) + "|" + "|".join(goal)

In [4]:
# sample input: {"domain": "blockworld", "init": [ "(on-table a)", "(on-table b)", "(clear a)", "(clear b)", "(arm-empty)" ], "goal": [ "(on a b)" ]}
usr_input = input("Enter Input: ")

Enter Input: {"domain": "blockworld", "init": [ "(on-table a)", "(on-table b)", "(clear a)", "(clear b)", "(arm-empty)" ], "goal": [ "(on a b)" ]}


In [5]:
usr_planning_domain, usr_problem = caseRepnGen(usr_input) #where problem P = (I,G)

### Method 1
##### The user enters the planning domain and problem = (init, goal). Method 1 then uses levenshtein distance as a metric to find similar plans between user's problem string and the problem strings existing in the case library for similar planning domains.

In [18]:
#function def for computing the normalized levenshtein similarity (1- normalized levenshtein distance)
def match(str1,str2):
    d = editdistance.eval(str1, str2)
    #return enchant.utils.levenshtein(str1, str2)
    return 1-(d/max(len(str1), len(str2)))

In [7]:
cases = file["cases"]
problem_metadata = ['initial','goal'] #metadata considered from the cases library

#creating the concatenated attributes of the metadata 
problem = []
for i in range(1,len(cases)+1):
    
    item = cases[str(i)]
    string = ""
    for x in problem_metadata:
        temp_List = item[x]
    
        if type(temp_List) == list:
            for j in temp_List:
                string = string + j + '|'
        else:
            string = string + temp_List + '|'
            
    problem.append(string)

In [8]:
#showing sample problem
problem[0]

'(on-table a)|(on-table b)|(clear a)|(clear b)|(arm-empty)|(on a b)|'

In [24]:
#lesser the normalized levenshtein distance between two strings, the more similar they are
ranked_cases = list()
for key in file['cases'].keys():
    if(file['cases'][key]['Planning_Domain'] == usr_planning_domain):
        distance = match(usr_problem,problem[int(key)-1])
        if distance >= 0.7:
            ranked_cases.append((distance,file['cases'][key]['Index']))
            
sorted(ranked_cases,key=itemgetter(0))

[(0.9850746268656716, 'blockworld-plans/plan_1.txt')]

### Method 2
##### The user enters the planning domain and problem = (init, goal). Method 2 considers the problem as a set and finds matching plans using jaccard distance as a metric.

In [41]:
#defining jaccard distance - similarity metric
def jaccard(list1, list2):
    #print(list1)
    #print(list2)
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [42]:
cases = file["cases"]

problem_metadata = ['initial','goal'] #considered attributes

#creating a set of attributes for each case
problem_set = []
for i in range(1,len(cases)+1):
    
    item = cases[str(i)]
    string = []
    for x in problem_metadata:
        temp_List = item[x]
    
        if type(temp_List) == list:
            for j in temp_List:
                string.append(j)
        else:
            string.append(temp_List)
            
    problem_set.append(string)

In [43]:
#example of the create problem set
problem_set[0]

['(on-table a)',
 '(on-table b)',
 '(clear a)',
 '(clear b)',
 '(arm-empty)',
 '(on a b)']

In [44]:
#converting user query (str) to a list for computing jaccard distance
usr_problem = usr_problem.split('|')

In [45]:
#retrieving similar plans based on jaccard distance when planning domain is matching with user query
#higher the jaccard score, the more the similarity between the two sets
ranked_cases = list()
for key in file['cases'].keys():
    if(file['cases'][key]['Planning_Domain'] == usr_planning_domain):
        sim_score = jaccard(usr_problem, problem_set[int(key)-1])
        if sim_score >= 0.8:
            ranked_cases.append((sim_score,file['cases'][key]['Index']))
sorted(ranked_cases,key=itemgetter(0))

[(1.0, 'blockworld-plans/plan_1.txt')]